In [1]:
include("Axis_Aligned_Box.jl");
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
    Θ::Nullable{Axis_Aligned_Box}   # data boxes B
    tab::Array{Int}
    c::Array{Int}
    Gₚ::Array{Float64}
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type,
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

type Mondrian_Tree
    root::Nullable{Mondrian_Node}
    leaves::Array{Mondrian_Node,1}
end

function Mondrian_Tree()
    T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}())
end

function Mondrian_Tree(N::Mondrian_Node)
    T = Mondrian_Tree(N,Array{Mondrian_Node,1}())
end

Mondrian_Tree

In [357]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    T = Mondrian_Tree(e)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    e.Θ = Θ
    e.tab = zeros(size(unique(D[2]),1))
    e.c = zeros(size(unique(D[2]),1))
    e.Gₚ = zeros(size(unique(D[2]),1))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, T)
    return T
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, tree)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        j.δ = d
        j.ζ = x
        j.τ = τₚ+E
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(0.0, [true,false,false])
        left.parent = j
        left.Θ = Θᴸ
        left.tab = zeros(size(j.tab))
        left.c = zeros(size(j.tab))
        left.Gₚ = zeros(size(unique(D[2]),1))
        right = Mondrian_Node(0.0, [true,false,false])
        right.parent = j
        right.Θ = Θᴿ
        right.tab = zeros(size(j.tab))
        right.c = zeros(size(j.tab))
        right.Gₚ=zeros(size(unique(D[2]),1))
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, tree)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, tree)
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(tree.leaves,j)
    end
end

function get_data_indices(Θ::Axis_Aligned_Box, D::Array{Float64,2})
    indices = []
    include = false
    for i in 1:size(D,1)
        for j in 1:size(Θ.Intervals,1)
            if (D[i,j] < Θ.Intervals[j,1] || D[i,j] > Θ.Intervals[j,2])
                include = false
                break
            end
            include = true
        end
        if (include)    
            push!(indices, i)
        end
    end
    return indices
end

function set_label_distribution!(j,Θ, D)
    indices = get_data_indices(Θ, D)
    for k in 1:length(unique(D[2]))
        j.Gₚ = length(D[2][D[2].==k])
    end
    j.Gₚ = j.Gₚ/(sum(j.Gₚ))
end

function initialize_posterior_leaf_counts!(Tree, D)
    X = D[1]
    Y = D[2]
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),X)
        y = Y[indices]
        for k in 1:length(leaf.c)
            leaf.c[k] = length(y[y.==k])
            leaf.tab[k] = min(leaf.c[k],1)
        end
    end
end

function initialize_posterior_counts!(Tree,D)
    initialize_posterior_leaf_counts!(Tree,D)
    for leaf in Tree.leaves
        j = leaf
        while true
            if j.node_type[2]==false
                for k in 1:length(j.c)
                    j.c[k] = get(j.left).tab[k]+get(j.right).tab[k]
                end
            end
            for k in 1:length(j.c)
                j.tab[k] = min(j.c[k],1)
            end
            if j.node_type[3]==true
                break
            else
                j = get(j.parent)
            end
        end
    end
end

function update_posterior_counts!(leaf,y)
    leaf.c[y] += 1
    l = leaf
    while true
        if l.tab[y] == 1
            return
        else
            if l.node_type[2] == false
                l.c[y] = get(l.left).tab[y]+get(l.right).tab[y]
            end
            l.tab[y] = min(l.c[y],1)
            if l.node_type[3] == true
                return
            else
                l = get(l.parent)
            end
        end
    end         
end

function update_posterior_counts_batch!(Tree,D)
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),D[1])
        for y in D[2][indices,:]
            update_posterior_counts(leaf,y)
        end
    end
end

function compute_predictive_posterior_distribution!(Tree, γ=1)
    J = [get(Tree.root)]
    while (size(J,1) != 0)
        j = shift!(J)
        if (j.node_type[3]==true)
            p = ones(length(j.c))./length(j.c)
            d = exp(-γ*(j.τ))
        else 
            d = exp(-γ*(j.τ-get(j.parent).τ))
            p = get(j.parent).Gₚ
            println(j.τ-get(j.parent).τ)
        end
        for k in 1:length(j.c)
            j.Gₚ[k] = (1/(sum(j.c)+1))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
        end
        if j.node_type[2] == false 
            push!(J, get(j.left))
            push!(J, get(j.right))
        end
    end
end

function expected_discount(nⱼ, Δⱼ,γ=1)
    Δ = rand(Truncated(Exponential(1/nⱼ),0,Δⱼ),1000)
    return mean(exp.(-γ*Δ))
end

function predict!(T,x, γ=1)
    j = get(T.root)
    not_sep = 1
    s = zeros(size(j.c,1))
    while true
        if (j.node_type[3] == true)
            Δⱼ = j.τ
        else
            Δⱼ = j.τ - get(j.parent).τ
        end
        nⱼ=0
        for d in size(x,1)
            nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
        end
        pⱼ = 1-exp(Δⱼ*nⱼ)
        if pⱼ > 0
            print("yes")
            # x branches
            jₓ = Mondrian_Node()
            if (j == get(j.parent).left)
                get(j.parent).left = jₓ
            else
                get(j.parent).right = jₓ
            end
            jₓ.parent = get(j.parent)
            j.parent = jₓ
            jₓ.left = j
            jₓ.right = Mondrian_Node()
            d = expected_discount(nⱼ, Δⱼ, γ)
            for k in 1:length(j.c)
                jₓ.c[k] = min(j.c[k],1)
            end
            jₓ.tab = jₓ.c
            for k in 1:length(jₓ.c)
                jₓ.Gₚ[k] = 1/(sum(jₓ.c)+1)*(jₓ.c[k] - d*jₓ.tab[k]+d*sum(jₓ.tab)*get(jₓ.parent).Gₚ[k])
            end
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*jₓ.Gₚ[k]
            end
        end
        if j.node_type[2] == true
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*j.Gₚ[k]
            end
            return s
        else
            not_sep = not_sep*(1-pⱼ)
            if x[get(j.δ)] <= get(j.ζ)
                j = get(j.left)
            else
                j = get(j.right)
            end
        end
    end
end

predict! (generic function with 2 methods)

In [358]:
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=4
X, Y = Fakedata(1000,d);
D = (X,Y.+1)

([-0.533808 1.24338 -1.59306 -0.199754; -0.369395 -1.13766 -0.524936 -1.40331; … ; 2.55873 1.56178 -1.731 -0.0907218; -0.381563 -0.0531876 -2.23915 0.890055], [1, 1, 1, 1, 1, 2, 1, 2, 1, 2  …  1, 1, 1, 2, 1, 2, 1, 2, 2, 1])

In [359]:
MT = Sample_Mondrian_Tree(100,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

LoadError: [91mInterruptException:[39m

In [306]:
using MLBase
pred = []
for i in 1:size(D[1],1) 
    p = predict(MT,D[1][i,:],10*size(D[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.732
[2, 1]


In [310]:
for leaf in MT.leaves
    println(leaf.Gₚ)
end

[0.590012, 0.407543]
[0.582034, 0.400216]


In [344]:
include("load_titanic.jl")
x_train, y_train, x_test, y_test = load();
D = (x_train, y_train.+1)
D_test = (x_test,y_test.+1)
MT = Sample_Mondrian_Tree(0.04,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

0.0004516127726425372
0.001514966950853042
0.00034606414741844473
0.0005231491256373131
0.005873881820908225
0.0004826517889197626
4.4874690615125286e-5
0.004083574653304214
0.0016903512495446394
0.00047141307203269617
8.661189082918679e-5
0.00048779355659443226
0.004723557695377872
0.0005039844685978587
0.00037414598393465726
0.0001430709152963382
0.00012380226588455927
0.001007837562200832
0.015835158430979532
0.0016222978388029858
0.0020305809785556
0.0009059709114985328
0.002039208749081386
0.0015957314381502465
0.0014652878442510363
0.003289871155025095
0.00802804730107922
0.0003766033394767892
0.0020977460526317286
0.001078201800652308
0.002604872018031664
0.0012864597398570086
0.0008990283456167211
0.0025704845675675156
1.7915027529058318e-5
0.0021907236062646326
0.0021391343901642074
0.0031026079465695036
0.00013082728120063225
0.009607253811886637
0.000226847751872334
0.0032862876844940558
0.003751824231048329
7.476568159572958e-5
0.000813186445375761
0.00017534592630346138
0.

0.0038302220803193045
0.001697307197477748
0.0022261141265275727
0.000583718653390063
0.0079910067296169
0.005035033986963481
0.015988726390252755
0.010753559639132656
0.005064907384260797
0.001701026832296209
0.0025751327161632268
0.012903517139123091
0.003118992534530256
0.012420264953677207
0.0030324292507398036
0.003503774550401465
0.009234098423311691
0.0010221252959795955
0.0036781663448399917
0.000887018055000131
0.0049192671789307155
0.007095787747530306
0.002513969688890693
0.00422865319900966
0.0014607598856767184
0.012089982627220738
0.002246743225411749
0.0013197265339941477
0.0004281273323891062
0.003992161495233447
0.0009436567248709291
0.0010425677263977409
0.00022416796944595314
0.004996048950415033
0.002386586460027219
0.0011608439287375209
0.0064018060391905236
0.0064544657808506475
0.0064544657808506475
0.009400290107742973
0.0068493207559502806
0.013733642404383527
0.018771476087138323
0.007921393262124474
0.001159816971120841
0.011912907670819668
0.0103499141343387

0.00039209906826412344
0.01062513020952471
0.006229822221654125
0.012188123746005662
0.005806017967678809
0.010562271897165014
0.001274921145459601
0.001930833631055142
0.011497106394994225
0.00032864320158626503
0.004420821584856328
0.008703972635834133
0.0022993983132958626
0.0041120527910076185
0.0041120527910076185
0.0073984248275664535
0.0073984248275664535
0.003235029297223982
0.003235029297223982
0.0035327539155965842
0.0035327539155965842
0.004674760144308586
0.004674760144308586
0.002187042554966566
0.01620167559720701
0.00986089478667258
0.00986089478667258
0.01926928278720714
0.00024168490070820142
0.01155828553828226
0.01155828553828226
0.011433884679900613
0.00041039092696201415
0.00698199498529271
0.005292743669714734
0.008586398873794228
0.013838939075721307
0.003091460631377635
0.0024563525564212443
0.010055577804905532
0.003417251211963847
0.0006844347208575276
0.00399508508093379
0.02288536468176689
0.011072054020723102
0.00465406802567573
0.017711380156238
0.00427325

0.0017042971052213737
0.0017042971052213737
0.0015075270832150198
0.0015075270832150198
0.0017962484067931411
0.007160603966417163
0.0037006097131037363
0.0022045687673365386
0.005017211321050451
0.0025460260461823148
0.014933982628729681
0.010867080670889536
0.0021638826220237856
0.0020020490121298255
0.0032241373304390183
0.0002573466967104865
0.004646913528929882
0.0007208789519218065
0.00434114666239839
0.008435164216408962
0.005169277912061041
0.01629253586331843
0.004376505763757414
0.015258566089045068
0.0025275155352609477
0.0017046621142803933
0.0027905708666605064
0.002900317097290741
0.014476209082128915
0.0045599975192445324
0.004087993833187309
0.004087993833187309
0.005359486977886747
0.005359486977886747
0.0020121650385076795
0.0020121650385076795
0.0020141366365676525
0.0009921875256211354
0.005402039693169068
0.00029739429152168123
0.0030182566060348096
0.002301797673353312
0.0008903420593689496
0.002614893736028835
0.0018985682025850787
0.01266504794744365
0.002973707

0.00030872911711329713
0.005773265481520497
0.008221939444501964
0.02570403161411938
0.01135945821046198
0.009945049306720669
0.0037317439090906995
0.00654544849362718
0.001062686470712386
0.0005847091721745169
0.008873494990565762
0.002113071343450139
0.003016509616982294
0.028439562461211706
0.016910843716583628
0.02697041110494184
0.003161428090783234
0.001366114204386145
0.0016141097994340301
4.282706129212291e-5
0.005746854644057801
0.017781503198639536
0.0018486516336567067
0.00475298930312814
0.0005504098469852111
0.016130671639648833
0.0018889715987573698
0.0006549828769843672
0.00018317591958725427
0.01798963286162273
0.01798963286162273
0.0015111866637231384
0.001997792799260406
0.015437304259598529
0.022041590271587218
0.002995903404020197
0.00674220449952722
0.002641649476508039
0.005797567423850309
0.002647642281890728
0.009944351185798912
0.012409702123141835
0.0064630694626618725
0.0013267994188920287
0.00914286898849271
0.001141336329528073
0.008095682697814867
0.001262

...

0.011605024028372851
0.011605024028372851
0.014215180922785974
0.014215180922785974
0.00038070306573709245
0.00038070306573709245
0.002076550453637159
0.007657640324029061
0.0015849623346506636
0.006029542545959485
0.00031298006105785064
0.00031298006105785064
1.452127209536358e-5
0.013859544782047858
0.01159488547840819
0.01159488547840819
0.0009227015350528403
0.0009227015350528403
0.003225599814258767
0.003225599814258767
0.007874328061392048
0.0031418830838415047
0.0022220695881824473
0.011663181887628514
0.002305263507021614
0.002305263507021614
0.005021214279065114
0.000952066737088815
0.000752335325127737
0.003210181872627775
0.004851587955028706
0.0026082071323839176
0.0013137566174013585
0.011099836448543201
0.005253773686223234
0.005253773686223234
0.0047819811994937525
0.0047819811994937525
0.012287087304230459
0.010832504631582373
0.002538030133760364
0.005981896058224138
0.0071458767773749585
0.0071458767773749585
0.00550262734170693
0.0008167382661210532
0.006267214337378

0.0009553280185168392
0.015516965432257127
0.012686698647933092
0.0012860453674678196
0.001240905948639924
0.008842465402270704
0.008654771321945899
0.0053385199498595035
0.0049558688433936765
0.010058531745134675
0.006380727953102846
0.00437405124181281
0.002308540829854388
0.0028337114911235264
0.0028337114911235264
0.00350577029299182
0.008848024432989137
0.0031292271009218733
0.004803657726382839
0.0015210348528401363
0.0015210348528401363
0.003699841088051374
0.0008083126573886143
0.002477005656844271
0.002477005656844271
0.0001985562763971993
0.0001985562763971993
0.0013022020145687443
0.0013022020145687443
0.0017163462907442972
0.0017163462907442972
0.0007614148887289465
0.0007614148887289465
0.004037583460102551
0.004899476323521439
0.0074969629851431505
0.0074969629851431505
0.005330909372632149
0.005330909372632149
0.0048869874679803825
0.01183813700632681
0.0016874112576232836
0.0016874112576232836
0.0024360802562183106
0.0039408052327734935
0.002880624167210008
0.0028806241

0.0051734353311996065
0.006234549358430354
0.0006918434407108751
0.0016700125415952602
0.0018730163284992538
0.0018730163284992538
0.002037465222911511
0.002037465222911511
0.0002416627149615866
0.0002416627149615866
0.001547923699973315
0.0005224418628146585
0.0012196889191853147
0.0014901294284435096
0.0036925963267992745
0.0029603579216195414
0.0003679909590992708
0.0003679909590992708
0.0012428171369868388
0.0012428171369868388
0.005088773496281555
0.00018453792735635688
0.00040788199590430924
0.00040788199590430924
0.0024225557496280192
0.0024225557496280192
0.0006552966770338245
0.0006552966770338245
0.005785973706404983
0.005581857698148965
0.002386464777181227
0.002386464777181227
0.006055824341498706
6.739810408481345e-5
0.002035394911602123
0.0012173854248931568
0.002600607136439982
0.002600607136439982
0.0003712523413913066
0.0003712523413913066
0.0005911052721980434
0.0005911052721980434
0.004739618322306023
0.004739618322306023
0.001199837933665071
0.001199837933665071
0.0

6.55977494697646e-5
6.55977494697646e-5
0.003219877319386555
0.003219877319386555
0.0033167363048835635
0.0033167363048835635
0.005937578299062524
0.005937578299062524
0.0018932689133546454
0.006347055424494888
0.000567603859671674
0.000567603859671674
0.0021092594316333355
0.0028073034403446814
0.0025513850858591564
0.0025513850858591564
0.0001900294975329328
0.0005578228374342534
0.00013083419586927547
0.0018064078058687288
0.0008970017818150239
0.0008970017818150239
0.006072448622688603
0.006072448622688603
0.0036681020648174772
0.0036681020648174772
0.0035356051544517056
0.001338267852999675
0.00015731176173650746
0.00015731176173650746
0.0017510264157581668
0.0017510264157581668
0.0007518931100818985
0.0007518931100818985
0.001666201810451337
0.00476280267608533
0.000706971598104629
0.000706971598104629
0.0038509954074291974
0.0038509954074291974
0.0037780328452720544
0.0037638037179548098
0.006044490986193488
0.00014154204300025064
0.0002082633887277724
0.0002082633887277724
0.00

0.009250925540657068
0.009250925540657068
0.0007044579543745502
0.0004132711669287831
0.0058356829710250815
0.0058356829710250815
0.008715929834796955
0.010575631311362837
0.0031408841517011274
0.0031408841517011274
0.00371619864918616
0.0021819577203575913
0.0004241619224334206
0.0022409076474935313
0.0007512208123263836
0.0007512208123263836
0.007366042474546013
0.01739615864254716
0.004006042776860903
0.00981648822898817
0.0019800837010802916
0.0018927547336706832
0.008138967063125904
0.0025402957240954135
0.005932783508434777
0.0003245778339711
0.0003365026167172769
0.0003365026167172769
0.012588062185602378
0.012588062185602378
0.005443238156447351
0.005443238156447351
0.009876785452099739
0.0016378567959912035
0.001978957697508367
0.00027171662690890497
0.00017451110638669332
0.00017451110638669332
0.005629115724532965
0.002066801600678181
0.00267431539410231
0.00267431539410231
0.0014408534645943688
0.0014408534645943688
0.0004591881048708682
0.006055352617147444
0.0067829959776

In [345]:
using MLBase
pred = []
for i in 1:size(D_test[1],1) 
    p = predict(MT,D_test[1][i,:],10*size(D_test[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D_test[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.55
[2, 1]


In [287]:
a.Intervals

LoadError: [91mtype Mondrian_Node has no field Intervals[39m

In [43]:
D[1][1,2] < a.Intervals[2,1] || D[1][1,2] > a.Intervals[2,2]

false

In [40]:
D[1,2]

LoadError: [91mMethodError: no method matching getindex(::Tuple{Array{Float64,2},Array{Int64,1}}, ::Int64, ::Int64)[39m

In [22]:
Y = [1,2,2,2,1]

5-element Array{Int64,1}:
 1
 2
 2
 2
 1

In [24]:
size(Y[Y.==1],1)

2

In [16]:
ones(2)

2-element Array{Float64,1}:
 1.0
 1.0

In [17]:
exp(2)

7.38905609893065

In [19]:
sum([1,2,3])

6